In [2]:
## S&P 500 STOCK CLUSTERING AND NEURAL NETWORK ANALYSIS ##
# This script downloads the most recent listing of stock tickers of the S&P 500 from Wikipedia
# It then downloads the basic stock prices dating back to January 1st, 2015
# These data points are aggreagated into basic average summaries as well as processed for neural network analysis.
# At its most fundimental level, the algorithm uses the previous n days of adjusted close data to predict a rise
# or fall in price the following day/week/month.

# It is important to note that the S&P 500 does not mimic the behavior of all stocks. Separately, the metrics by which
# we evaluate our model are constantly improving. At this point, there is a theoretical lower-bound of 50% accuracy:
# did the stock go up or down. However we also seek to gauge how much the stock increased or decreased by increasing
# the number of neurons in the output layer or by running an entirely separate anlaysis--such a discussion is ongoing.

In [3]:
import datetime
import urllib.request
import pandas as pd
from pandas import DataFrame
from pandas.io.data import DataReader
import re

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/io/data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [4]:
# COLLECT STOCK TICKERS
wiki_page = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
wiki_page = str(wiki_page.read())

In [5]:
ticker_locations_part_one = [m.start() for m in re.finditer('XNYS', wiki_page)]
ticker_locations_part_two = [m.start() for m in re.finditer('.com/symbol/', wiki_page)]
all_tickers = []
for t in range(0,len(ticker_locations_part_one)): # t=1
    ticker = wiki_page[ticker_locations_part_one[t]:ticker_locations_part_one[t]+15]
    ticker = re.search('XNYS:(.+?)>',ticker).group(1).replace('"','')
    all_tickers.append(ticker)
for t in range(0,len(ticker_locations_part_two)):
    ticker = wiki_page[ticker_locations_part_two[t]:ticker_locations_part_two[t]+25]
    ticker = re.search('com/symbol/(.+?)>',ticker).group(1).replace('"','')
    all_tickers.append(ticker)

In [6]:
symbols_list = all_tickers

symbols=[]
for ticker in symbols_list:
    try:
        r = DataReader(ticker, "yahoo", start=datetime.datetime(2014, 12, 30))
        # add a symbol column
        r['Symbol'] = ticker 
        symbols.append(r)
    except:
        print(ticker, "has been renamed")
# concatenate all the dfs
df = pd.concat(symbols)
#define cell with the columns that i need
cell= df[['Symbol','Open','High','Low','Adj Close','Volume']]
#changing sort of Symbol (ascending) and Date(descending) setting Symbol as first column and changing date format
cell = cell.reset_index().sort(['Symbol', 'Date'], ascending=[1,0]).set_index(['Symbol'])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [7]:
last_close = []
five_day_avg = []
one_month_avg = []
two_month_avg = []
three_month_avg = []
six_month_avg = []
twelve_month_avg = []

last_close_vol = []
five_day_avg_vol = []
one_month_avg_vol = []
two_month_avg_vol = []
three_month_avg_vol = []
six_month_avg_vol = []
twelve_month_avg_vol = []

for i in range(0,len(symbols_list)):
    ticker = symbols_list[i]
    subset = cell.loc[ticker]

    last_close.append(subset.iloc[2,:]["Adj Close"])
    five_day_avg.append(subset.iloc[2:7,:]["Adj Close"].mean())
    one_month_avg.append(subset.iloc[2:32,:]["Adj Close"].mean())
    two_month_avg.append(subset.iloc[2:62,:]["Adj Close"].mean())
    three_month_avg.append(subset.iloc[2:92,:]["Adj Close"].mean())
    six_month_avg.append(subset.iloc[2:182,:]["Adj Close"].mean())
    twelve_month_avg.append(subset.iloc[2:365,:]["Adj Close"].mean())

    last_close_vol.append(subset.iloc[2,:]["Volume"])
    five_day_avg_vol.append(subset.iloc[2:7,:]["Volume"].mean())
    one_month_avg_vol.append(subset.iloc[2:32,:]["Volume"].mean())
    two_month_avg_vol.append(subset.iloc[2:62,:]["Volume"].mean())
    three_month_avg_vol.append(subset.iloc[2:92,:]["Volume"].mean())
    six_month_avg_vol.append(subset.iloc[2:182,:]["Volume"].mean())
    twelve_month_avg_vol.append(subset.iloc[2:365,:]["Volume"].mean())

ticker_data = {'Ticker':ticker,'Last Close':last_close,'Five Day Avg':five_day_avg,'One Month Avg':one_month_avg,
               'Two Month Avg':two_month_avg,'Three Month Avg':three_month_avg,'Six Month Avg':six_month_avg,
               'Twelve Month Avg':twelve_month_avg,'Last Close Vol':last_close_vol,'Five Day Avg Vol':five_day_avg_vol,
               'One Month Avg Vol':one_month_avg_vol,'Two Month Avg Vol':two_month_avg_vol,
               'Three Month Avg Vol':three_month_avg_vol,'Six Month Avg Vol':six_month_avg_vol,
               'Twelve Month Avg Vol':one_month_avg_vol}
ticker_outputs = DataFrame(data=ticker_data, index=symbols_list)
ticker_outputs = ticker_outputs[['Last Close','Five Day Avg','One Month Avg','Two Month Avg','Three Month Avg',
                                'Six Month Avg','Twelve Month Avg','Last Close Vol','Five Day Avg Vol',
                                'One Month Avg Vol','Two Month Avg Vol','Three Month Avg Vol','Six Month Avg Vol',
                                'Twelve Month Avg Vol']]

In [8]:
ticker_outputs

,Last Close,Five Day Avg,One Month Avg,Two Month Avg,Three Month Avg,Six Month Avg,Twelve Month Avg,Last Close Vol,Five Day Avg Vol,One Month Avg Vol,Two Month Avg Vol,Three Month Avg Vol,Six Month Avg Vol,Twelve Month Avg Vol
MMM,166.229996,169.354001,173.325333,176.070849,175.817996,169.546237,158.105565,3826200,2176520.0,1.999397e+06,1.690383e+06,1.808246e+06,1.893619e+06,1.999397e+06
ABT,40.189999,40.370000,41.415165,42.352817,41.839862,40.540019,42.316572,6598400,9779820.0,8.175700e+06,8.832002e+06,9.510171e+06,9.193823e+06,8.175700e+06
ABBV,61.549999,61.367999,62.319863,63.650359,63.155062,60.390217,59.652779,6432800,4937900.0,5.627473e+06,6.069707e+06,6.617741e+06,7.392722e+06,5.627473e+06
ACN,115.269997,115.906000,114.823843,113.697914,113.499321,111.668596,105.124865,1456400,1897240.0,2.805680e+06,2.358627e+06,2.427122e+06,2.411398e+06,2.805680e+06
AYI,233.889999,239.737998,251.306379,261.411528,259.986305,246.942779,223.706795,526100,438440.0,4.882333e+05,3.523133e+05,3.502267e+05,4.522417e+05,4.882333e+05
AAP,143.809998,142.073999,146.818827,153.935781,156.679169,155.069409,161.142931,1117800,1146980.0,9.321933e+05,1.044258e+06,1.007526e+06,9.897500e+05,9.321933e+05
AES,12.020000,12.052000,12.302333,12.292667,12.260523,11.509913,11.122431,3173300,4013400.0,5.523310e+06,5.095838e+06,5.048466e+06,5.001703e+06,5.523310e+06
AET,111.639999,111.379999,113.307578,115.472488,116.258531,113.837883,112.396121,1988700,2155140.0,2.042510e+06,1.873142e+06,2.496933e+06,2.417822e+06,2.042510e+06
AFL,69.639999,69.867999,71.486000,72.115865,71.954491,68.269768,63.833840,1313100,1666380.0,1.567547e+06,1.601377e+06,1.752660e+06,1.980386e+06,1.567547e+06
AMG,141.770004,141.722000,144.790334,143.948167,143.841000,151.016056,167.212369,259700,396220.0,4.635300e+05,4.303800e+05,5.088911e+05,5.378911e+05,4.635300e+05


In [9]:
ticker_outputs.loc['aapl']

Last Close              1.182500e+02
Five Day Avg            1.173360e+02
One Month Avg           1.149320e+02
Two Month Avg           1.111606e+02
Three Month Avg         1.064747e+02
Six Month Avg           1.030465e+02
Twelve Month Avg        1.079839e+02
Last Close Vol          4.812900e+07
Five Day Avg Vol        2.780416e+07
One Month Avg Vol       3.867253e+07
Two Month Avg Vol       3.469855e+07
Three Month Avg Vol     3.523213e+07
Six Month Avg Vol       3.666131e+07
Twelve Month Avg Vol    3.867253e+07
Name: aapl, dtype: float64

In [12]:
cell.loc['A']

,Date,Open,High,Low,Adj Close,Volume
Symbol,,,,,,
A,2016-10-27,43.970001,44.189999,43.349998,43.380001,2392800
A,2016-10-26,43.419998,43.820000,43.240002,43.369999,2146200
A,2016-10-25,45.259998,45.590000,43.169998,43.509998,3949500
A,2016-10-24,45.880001,45.950001,45.639999,45.849998,1666400
A,2016-10-21,45.669998,45.669998,45.290001,45.570000,1011300
A,2016-10-20,46.000000,46.220001,45.650002,46.049999,1493600
A,2016-10-19,45.410000,46.150002,45.340000,46.020000,1769000
A,2016-10-18,45.700001,45.830002,45.270000,45.439999,1949100
A,2016-10-17,45.080002,45.470001,45.000000,45.150002,1160300
